# Set up

In [ ]:
project_folder = "/content/drive/MyDrive/water_quality_predictor"

from google.colab import drive
drive.mount('/content/drive')

%cd "{project_folder}"
!git pull

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/water_quality_predictor
Already up to date.


# Cài đặt môi trường

In [ ]:
!git pull
!pip install -r requirements.txt

Already up to date.
Obtaining file:///content/drive/MyDrive/water_quality_predictor (from -r requirements.txt (line 17))
  Preparing metadata (setup.py) ... done
  Attempting uninstall: classifier
    Found existing installation: classifier 0.0.0
    Uninstalling classifier-0.0.0:
      Successfully uninstalled classifier-0.0.0
  Running setup.py develop for classifier


# Thư viện

In [ ]:
import os
import matplotlib.pyplot as plt
from Mylib import myfuncs
from sklearn.model_selection import train_test_split
import pandas as pd


# Read the dataset


In [ ]:
df = pd.read_csv("data/Water Quality Prediction 300k.csv")
df.head()


,Unnamed: 0,Index,pH,Iron,Nitrate,Chloride,Lead,Zinc,Color,Turbidity,...,Chlorine,Manganese,Total Dissolved Solids,Source,Water Temperature,Air Temperature,Month,Day,Time of Day,Target
0,597538,597538,9.522424,4.792145e-01,6.687850,359.702547,5.660000e-80,1.751097,Near Colorless,0.872356,...,2.141352,0.113169,512.778184,Lake,27.221744,58.985360,May,22.0,17.0,0
1,2274,2274,6.866225,1.095113e-03,2.481730,178.385858,1.610000e-126,0.410191,Near Colorless,0.294705,...,3.165534,0.000008,312.133243,Well,14.080138,54.410223,September,27.0,22.0,0
2,729626,729626,7.521999,7.055721e-02,11.359830,350.564642,5.290000e-158,2.477065,Yellow,0.438921,...,3.776310,0.000208,414.269400,Aquifer,13.973573,65.334872,November,25.0,22.0,1
3,75485,75485,6.026060,4.200000e-08,3.808772,169.217390,5.510000e-152,0.021406,Yellow,0.499565,...,3.721139,0.008046,306.345464,Ground,24.123817,50.562887,December,16.0,10.0,0
4,207631,207631,7.836850,4.196171e-02,7.905958,160.114303,3.460000e-64,0.759456,Colorless,0.094661,...,3.283765,0.007605,196.749287,Well,17.687728,43.722289,September,2.0,2.0,0


In [ ]:
df.columns

Index(['Unnamed: 0', 'Index', 'pH', 'Iron', 'Nitrate', 'Chloride', 'Lead',
       'Zinc', 'Color', 'Turbidity', 'Fluoride', 'Copper', 'Odor', 'Sulfate',
       'Conductivity', 'Chlorine', 'Manganese', 'Total Dissolved Solids',
       'Source', 'Water Temperature', 'Air Temperature', 'Month', 'Day',
       'Time of Day', 'Target'],
      dtype='object')

In [ ]:
df = df.drop(columns = ['Unnamed: 0', 'Index'])

df.columns

Index(['pH', 'Iron', 'Nitrate', 'Chloride', 'Lead', 'Zinc', 'Color',
       'Turbidity', 'Fluoride', 'Copper', 'Odor', 'Sulfate', 'Conductivity',
       'Chlorine', 'Manganese', 'Total Dissolved Solids', 'Source',
       'Water Temperature', 'Air Temperature', 'Month', 'Day', 'Time of Day',
       'Target'],
      dtype='object')

# Xem xét cột target

In [ ]:
df['Target'].isnull().sum()

0

In [ ]:
df.shape

(314573, 23)

In [ ]:
for col in df.columns:
  print(f"{col} -> {df[col].unique()}")

pH -> [9.52242354 6.86622453 7.52199856 ... 8.0088107  8.39152498 8.86302209]
Iron -> [4.79214469e-01 1.09511300e-03 7.05572080e-02 ... 3.23597000e-04
 4.12435000e-04 4.54404115e-01]
Nitrate -> [ 6.68784951  2.4817297  11.35982981 ...  6.81304519 10.26698032
  3.41434743]
Chloride -> [359.7025471  178.3858581  350.5646425  ...  47.24457825 129.1511748
 187.6658915 ]
Lead -> [5.66e-080 1.61e-126 5.29e-158 ... 6.46e-184 8.81e-051 3.59e-174]
Zinc -> [1.75109692 0.4101912  2.47706538 ... 2.44526913 5.91089939 0.75695351]
Color -> ['Near Colorless' 'Yellow' 'Colorless' 'Faint Yellow' 'Light Yellow' nan]
Turbidity -> [0.872356   0.29470465 0.43892147 ... 0.43660748 1.23516529 1.58217681]
Fluoride -> [0.40768929 0.17468591 0.72469701 ... 0.88835445 0.08315082 0.07205221]
Copper -> [0.43071102 0.47069234 1.05540144 ... 0.31186559 0.96347775 0.62161666]
Odor -> [0.77238123 1.83986026 3.17914161 ... 1.99317583 2.73417311 0.09728051]
Sulfate -> [ 86.27791582  80.59789237  79.82708409 ...  72.2468

In [ ]:
df[['Month', 'Day', 'Time of Day']].isnull().mean() * 100

,0
Month,1.581827
Day,1.671790
Time of Day,1.911798


In [ ]:
df = df.dropna(subset=['Month', 'Day', 'Time of Day'])

df[['Month', 'Day', 'Time of Day']].isnull().mean() * 100

,0
Month,0.0
Day,0.0
Time of Day,0.0


In [ ]:
for col in ['Month', 'Day', 'Time of Day']:
  print(f"{col} -> {df[col].unique().tolist()}")

Month -> ['May', 'September', 'November', 'December', 'February', 'October', 'August', 'March', 'June', 'April', 'July', 'January']
Day -> [22.0, 27.0, 25.0, 16.0, 2.0, 29.0, 12.0, 6.0, 24.0, 30.0, 9.0, 21.0, 3.0, 7.0, 23.0, 1.0, 18.0, 10.0, 11.0, 15.0, 31.0, 26.0, 5.0, 8.0, 19.0, 28.0, 14.0, 13.0, 20.0, 17.0, 4.0]
Time of Day -> [17.0, 22.0, 10.0, 2.0, 16.0, 14.0, 8.0, 0.0, 15.0, 23.0, 12.0, 13.0, 9.0, 5.0, 3.0, 21.0, 19.0, 4.0, 20.0, 1.0, 11.0, 18.0, 6.0, 7.0]


In [ ]:
df.shape

(298624, 23)

In [ ]:
month_dict = {
    "January": 1,
    "February": 2,
    "March": 3,
    "April": 4,
    "May": 5,
    "June": 6,
    "July": 7,
    "August": 8,
    "September": 9,
    "October": 10,
    "November": 11,
    "December": 12,
}

df['Month'] = df['Month'].apply(lambda item: month_dict[item])

df['Month'].unique().tolist()

[5, 9, 11, 12, 2, 10, 8, 3, 6, 4, 7, 1]

In [ ]:
df['Day'] = df['Day'].astype('int')
df['Time of Day'] = df['Time of Day'].astype('int')
df['taken_time'] = df[['Month', 'Day', 'Time of Day']].astype('str').agg('-'.join, axis=1)
df['taken_time'] = pd.to_datetime(df['taken_time'], format='%m-%d-%H')

df['taken_time'].head()

,taken_time
0,1900-05-22 17:00:00
1,1900-09-27 22:00:00
2,1900-11-25 22:00:00
3,1900-12-16 10:00:00
4,1900-09-02 02:00:00


In [ ]:
df = df.sort_values(by = 'taken_time', ascending = True)

df.head()

,pH,Iron,Nitrate,Chloride,Lead,Zinc,Color,Turbidity,Fluoride,Copper,...,Manganese,Total Dissolved Solids,Source,Water Temperature,Air Temperature,Month,Day,Time of Day,Target,taken_time
116899,7.034305,3.510000e-14,2.859233,132.125438,6.540000e-235,2.152380,Light Yellow,1.377299,1.551360,0.095312,...,0.000332,504.928844,Reservoir,7.116495,72.255851,1,1,0,1,1900-01-01
155434,6.599581,3.990000e-07,5.371393,199.138779,1.330000e-58,1.552503,Near Colorless,0.071201,1.776722,0.420041,...,0.021912,403.644429,Spring,23.035464,26.433255,1,1,0,0,1900-01-01
165968,7.670866,9.310000e-06,1.156363,245.548989,7.130000e-66,0.608690,Colorless,0.428635,0.506733,0.234863,...,0.002062,391.642190,Spring,5.728718,54.360327,1,1,0,0,1900-01-01
150476,8.024941,8.664510e-02,3.146721,174.001989,2.460000e-113,1.151119,Colorless,0.583815,0.543565,0.169842,...,0.002520,53.432049,River,26.853255,71.579596,1,1,0,0,1900-01-01
255559,6.738499,3.550000e-11,3.481104,225.121948,5.470000e-05,1.494975,Faint Yellow,0.863283,1.961606,NaN,...,0.000029,157.238713,Well,9.778199,66.545395,1,1,0,0,1900-01-01


In [33]:
df.columns

Index(['pH', 'Iron', 'Nitrate', 'Chloride', 'Lead', 'Zinc', 'Color',
       'Turbidity', 'Fluoride', 'Copper', 'Odor', 'Sulfate', 'Conductivity',
       'Chlorine', 'Manganese', 'Total Dissolved Solids', 'Source',
       'Water Temperature', 'Air Temperature', 'Month', 'Day', 'Time of Day',
       'Target', 'taken_time'],
      dtype='object')

In [34]:
df = df.drop(columns = ['Month', 'Day', 'Time of Day'])

df.columns

Index(['pH', 'Iron', 'Nitrate', 'Chloride', 'Lead', 'Zinc', 'Color',
       'Turbidity', 'Fluoride', 'Copper', 'Odor', 'Sulfate', 'Conductivity',
       'Chlorine', 'Manganese', 'Total Dissolved Solids', 'Source',
       'Water Temperature', 'Air Temperature', 'Target', 'taken_time'],
      dtype='object')

# CHia dữ liệu thành tập train, tập val, tập test
- 60% -> training
- 20% -> val
- 20% -> test


In [35]:
train_size = 0.6
val_size = 0.2

num_train = int(train_size * len(df))
num_val = int(val_size * len(df))

df_train = df.iloc[:num_train, :]
df_val = df.iloc[num_train: num_train + num_val, :]
df_test = df.iloc[num_train + num_val:, :]

df_train.shape, df_val.shape, df_test.shape

((179174, 21), (59724, 21), (59726, 21))

# Lưu dữ liệu



In [36]:
data_folder = "artifacts/data_ingestion"
os.makedirs(data_folder, exist_ok=True)

myfuncs.save_python_object(os.path.join(data_folder, "raw_data.pkl"), df)
myfuncs.save_python_object(os.path.join(data_folder, "train_data.pkl"), df_train)
myfuncs.save_python_object(os.path.join(data_folder, "val_data.pkl"), df_val)
myfuncs.save_python_object(os.path.join(data_folder, "test_data.pkl"), df_test)